# Get historical market debt

* Query old blocks to get marketDebt for each market over time


In [1]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
import sqlite3
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [3]:
# data contract
with open('../abi/PerpsV2Data.json', 'r') as file:
    PerpsV2DataAbi = json.dumps(json.load(file))

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


## Get the market debt

In [4]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)

def getMarketDebt(lookback):
    this_block = w3.eth.block_number

    if lookback == 0:
        block_lookback = this_block
    elif lookback >= 0:
        block_lookback = -1 * lookback
    else:
        raise ValueError("Lookback must be >= 0")
    
    block = w3.eth.get_block(this_block - lookback)
    
    marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call(block_identifier=block_lookback)

    markets = [{
        'asset': market[1].decode().replace('\x00', ''),
        'marketDebt': w3.fromWei(market[7], unit='ether'),
    } for market in marketSummaries]

    df_markets = pd.DataFrame(markets)
    df_markets['marketDebt'] = df_markets['marketDebt'].astype(float)
    df_markets = df_markets.set_index('asset').transpose()
    df_markets['block'] = block['number']
    df_markets['timestamp'] = block['timestamp']

    return df_markets



In [6]:
df = pd.DataFrame()
for i in range(10):
    print(f'Running number {i}')
    df_this_block = getMarketDebt(i*100000)
    df = pd.concat([df, df_this_block], axis=0)

df

Running number 0
Running number 1
Running number 2
Running number 3
Running number 4
Running number 5
Running number 6
Running number 7
Running number 8
Running number 9


asset,sETH,sBTC,LINK,SOL,AVAX,AAVE,UNI,MATIC,APE,DYDX,...,EUR,ATOM,AXS,FLOW,FTM,NEAR,AUD,GBP,block,timestamp
marketDebt,6.801249e+06,5.997203e+06,243501.860505,466803.796857,343185.550370,2983.316343,96799.961713,479314.020277,55507.733370,25170.568979,...,27812.415707,27220.884917,1591.295669,7791.340609,42507.771830,3152.731976,1291.052916,2422.180091,83745676,1679809442
marketDebt,7.131890e+06,6.263659e+06,249266.581248,436187.266600,388582.350729,3034.912706,87562.926707,548058.387200,62038.870968,32369.165319,...,27587.079647,25410.028050,1368.368607,10592.196746,129453.223749,2600.202780,1290.448493,2426.643167,83645676,1679759064
marketDebt,7.457181e+06,6.934821e+06,219706.312593,433271.828140,293184.443854,2632.523406,87944.185811,366648.309261,62260.047513,36290.395432,...,27421.900388,22794.263054,765.819609,10700.336311,128007.137995,2606.999904,1291.492717,2429.553726,83545676,1679725387
marketDebt,8.063243e+06,7.615421e+06,178783.540994,686196.296286,265591.124437,2649.522141,102382.727359,448026.015756,66230.750869,37957.193046,...,24260.399116,17254.468908,487.839964,10687.202874,44143.577600,2362.940441,1288.483483,2468.089966,83445678,1679678828
marketDebt,7.985014e+06,8.507511e+06,347170.163422,703427.413374,283520.213419,2563.912340,102299.366070,487363.691631,66101.303708,28745.920139,...,23634.874967,15763.505882,442.567930,3999.086027,42633.281977,2187.617159,1322.863372,2141.538786,83345683,1679648928
marketDebt,8.066666e+06,8.572470e+06,380706.125593,672077.857810,281428.775428,2302.405311,101715.281695,453456.920030,64942.118670,28688.452220,...,21685.967489,14817.411080,390.380657,3849.276568,42119.965791,2025.808233,1350.574971,2017.587435,83245688,1679606423
marketDebt,8.204039e+06,8.327983e+06,351804.468595,653172.526313,322567.830187,2253.109938,89032.239167,444183.454173,64394.855851,39555.251150,...,140058.905124,49591.170283,2353.299388,4444.674458,52121.774137,1759.135193,1406.607424,2002.148877,83145688,1679570565
marketDebt,8.305777e+06,8.685561e+06,248857.686286,505393.671787,322712.369874,2327.860886,79797.137200,418347.759456,63997.585127,32422.749709,...,139911.013980,51938.079455,473.514986,10632.677321,40447.482981,1805.921145,1385.751135,2014.956794,83045688,1679533569
marketDebt,8.040460e+06,9.210137e+06,167200.245697,187805.646340,288404.396050,1739.249280,83574.661353,318556.025965,60030.910843,31895.986689,...,140033.550054,152842.522603,182.797162,4873.530929,44176.392986,1738.474828,1300.492697,2218.449555,82945690,1679505197
marketDebt,8.488442e+06,8.797530e+06,325653.881347,265207.617150,320784.114431,1371.516986,84177.085616,323150.472740,64888.442734,32302.106889,...,139721.717517,344702.885465,180.694668,4991.028518,48365.087418,1705.365892,1340.553392,2141.273858,82845694,1679469440


In [9]:
assets = [c for c in df.columns if c not in ['block', 'timestamp']]
df = df[['block', 'timestamp'] + assets].sort_values('timestamp')
df

asset,block,timestamp,sETH,sBTC,LINK,SOL,AVAX,AAVE,UNI,MATIC,...,XAU,XAG,EUR,ATOM,AXS,FLOW,FTM,NEAR,AUD,GBP
marketDebt,82845694,1679469440,8.488442e+06,8.797530e+06,325653.881347,265207.617150,320784.114431,1371.516986,84177.085616,323150.472740,...,139829.600480,97278.708341,139721.717517,344702.885465,180.694668,4991.028518,48365.087418,1705.365892,1340.553392,2141.273858
marketDebt,82945690,1679505197,8.040460e+06,9.210137e+06,167200.245697,187805.646340,288404.396050,1739.249280,83574.661353,318556.025965,...,144055.639962,98952.184476,140033.550054,152842.522603,182.797162,4873.530929,44176.392986,1738.474828,1300.492697,2218.449555
marketDebt,83045688,1679533569,8.305777e+06,8.685561e+06,248857.686286,505393.671787,322712.369874,2327.860886,79797.137200,418347.759456,...,135353.584241,101389.173643,139911.013980,51938.079455,473.514986,10632.677321,40447.482981,1805.921145,1385.751135,2014.956794
marketDebt,83145688,1679570565,8.204039e+06,8.327983e+06,351804.468595,653172.526313,322567.830187,2253.109938,89032.239167,444183.454173,...,140972.586815,100378.532650,140058.905124,49591.170283,2353.299388,4444.674458,52121.774137,1759.135193,1406.607424,2002.148877
marketDebt,83245688,1679606423,8.066666e+06,8.572470e+06,380706.125593,672077.857810,281428.775428,2302.405311,101715.281695,453456.920030,...,105802.876735,101290.583048,21685.967489,14817.411080,390.380657,3849.276568,42119.965791,2025.808233,1350.574971,2017.587435
marketDebt,83345683,1679648928,7.985014e+06,8.507511e+06,347170.163422,703427.413374,283520.213419,2563.912340,102299.366070,487363.691631,...,120161.647168,105516.149653,23634.874967,15763.505882,442.567930,3999.086027,42633.281977,2187.617159,1322.863372,2141.538786
marketDebt,83445678,1679678828,8.063243e+06,7.615421e+06,178783.540994,686196.296286,265591.124437,2649.522141,102382.727359,448026.015756,...,119790.731421,118071.650369,24260.399116,17254.468908,487.839964,10687.202874,44143.577600,2362.940441,1288.483483,2468.089966
marketDebt,83545676,1679725387,7.457181e+06,6.934821e+06,219706.312593,433271.828140,293184.443854,2632.523406,87944.185811,366648.309261,...,116096.357236,117986.117639,27421.900388,22794.263054,765.819609,10700.336311,128007.137995,2606.999904,1291.492717,2429.553726
marketDebt,83645676,1679759064,7.131890e+06,6.263659e+06,249266.581248,436187.266600,388582.350729,3034.912706,87562.926707,548058.387200,...,116003.870773,118021.156671,27587.079647,25410.028050,1368.368607,10592.196746,129453.223749,2600.202780,1290.448493,2426.643167
marketDebt,83745676,1679809442,6.801249e+06,5.997203e+06,243501.860505,466803.796857,343185.550370,2983.316343,96799.961713,479314.020277,...,115861.790456,118102.969204,27812.415707,27220.884917,1591.295669,7791.340609,42507.771830,3152.731976,1291.052916,2422.180091


In [12]:
df.melt(id_vars=['block', 'timestamp'], var_name='asset', value_name='market_debt')

,block,timestamp,asset,market_debt
0,82845694,1679469440,sETH,8.488442e+06
1,82945690,1679505197,sETH,8.040460e+06
2,83045688,1679533569,sETH,8.305777e+06
3,83145688,1679570565,sETH,8.204039e+06
4,83245688,1679606423,sETH,8.066666e+06
...,...,...,...,...
225,83345683,1679648928,GBP,2.141539e+03
226,83445678,1679678828,GBP,2.468090e+03
227,83545676,1679725387,GBP,2.429554e+03
228,83645676,1679759064,GBP,2.426643e+03
